# Librairie

In [30]:
import glob
import pandas as pd
import re
import requests
from datetime import date
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import NamedStyle
from bs4 import BeautifulSoup


# Chargement de fichier (pour test)

In [31]:
filepath = "Travail commun AO - Fichiers pour Sage"


In [32]:
dtype={'ETAB':object, 'JNAL':str, 'TYPE_PIECE':str, 'REF_PIECE':str,  'LIB_PIECE':str,
       'CPT_GENERAL':str, 'AUXILIAIRE_TIERS':object, 'LIBELLE_ECR':str, 'SENS':str,
        'MONTANT_TENUE_CPTE':object, 'MONTANT_TRANSAC':object, 'DEV':str, 'DATE_ECH':object, 'MODE_REG':object,
        'SECTION_ana':object, 'DEV_TC':object, 'CODE_LET':object, 'PROFIL_TVA':object, 'DETAIL_AFFAIRE':object,
         'TYPE_CPTE':object, 'OID_ECRITURE':object, 'CLASSE_CPTE':str}

In [33]:
batch = glob.glob("{}/CONTROLE*.xlsx".format(filepath))
batch

['Travail commun AO - Fichiers pour Sage\\CONTROLE_RDD_121_2022(1).xlsx',
 'Travail commun AO - Fichiers pour Sage\\CONTROLE_RDD_123_2022(2).xlsx',
 'Travail commun AO - Fichiers pour Sage\\CONTROLE_RDD_156_2022(1).xlsx']

In [34]:

file = 'Travail commun AO - Fichiers pour Sage\\CONTROLE_RDD_101_2022.xlsx'
file_test = 'Travail commun AO - Fichiers pour Sage\\test.xlsx'
test = pd.read_excel(
                      file_test,
                      #file,
                      dtype=dtype,decimal=','
                     )

In [35]:
str(test["DATE_PIECE"].dt.strftime('%d/%m/%Y')[0])

'01/01/2022'

In [36]:
[date for date in test["DATE_PIECE"].dt.strftime('%d/%m/%Y')]

['01/01/2022',
 '01/01/2022',
 '01/01/2022',
 '03/01/2022',
 '03/01/2022',
 '03/01/2022',
 '03/01/2020',
 '03/01/2022',
 '03/01/2022',
 '03/01/2022',
 '03/01/2022',
 '03/01/2022',
 '03/01/2022',
 '03/01/2022',
 '05/01/2022',
 '05/01/2022',
 '05/01/2022',
 '05/01/2022',
 '05/01/2022',
 '05/01/2022',
 '05/01/2022',
 '05/01/2022',
 '05/01/2022',
 '05/01/2022',
 '05/01/2022',
 '05/01/2022',
 '05/01/2024',
 '05/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '06/01/2022',
 '07/01/2022',
 '07/01/2022',
 '07/01/2022',
 '08/01/2022',
 '08/01/2022',
 '08/01/2022',
 '08/01/2022',
 '11/01/2022',
 '11/01/2022',
 '11/01/2022',
 '11/01/2022',
 '11/01/2022',
 '11/01/2022',
 '11/01/2022',
 '11/01/2022',
 '13/01/2022',
 '13/01/2022',
 '14/01/2022',
 '14/01/2022',
 '14/01/2022',
 '14/01/2022',
 '14/01/20

# fonctions par colonne

## ETAB

In [37]:
def ETAB(df):
    '''Aucune valeur vide, Aucune valeur "_ETAB",Aucune valeur "_ETAB0"'''
    col = "ETAB" 
    list_etab=[]
    list_etab0=[]
    list_na = df[df[col].isna()]["OID_ECRITURE"].tolist()
    for etab in df[col]:
        if etab =="_ETAB":
            list_etab.append(df[df[col]==etab]["OID_ECRITURE"].iloc[0])
        elif etab=="_ETAB0":
            list_etab0.append(df[df[col]==etab]["OID_ECRITURE"].iloc[0])
    if len(list_etab)+len(list_etab0)+len(list_na) :
        
        ETAB = pd.DataFrame.from_dict({"Contient des cellules vides" : list(set(list_na)),
                                   "Contient '_ETAB'" : list(set(list_etab)), 
                                   "Contient '_ETAB0'": list(set(list_etab0))}, orient="index")
        return ETAB 
    

## JNAL

In [38]:
def JNAL(df):
    '''Aucune valeur vide,Aucune valeur "_JNAL",Aucune valeur "0COMPTEBQ"'''
    col = "JNAL"
    list_jnal=[]
    list_0comptebq=[]
    list_na = df[df[col].isna()]["OID_ECRITURE"].tolist()
    for item in df[col]:
        if item =="_JNAL":
            list_jnal.append(df[df[col]==item]["OID_ECRITURE"].iloc[0])
        elif item=="0COMPTEBQ":
            list_0comptebq.append(df[df[col]==item]["OID_ECRITURE"].iloc[0])
    
    if len(list_jnal)+len(list_0comptebq)+len(list_na):
        JNAL = pd.DataFrame.from_dict({"Contient des cellules vides" : list(set(list_na)),
                                   "Contient '_JNAL'" : list(set(list_jnal)), 
                                   "Contient '0COMPTEBQ'": list(set(list_0comptebq))}, orient="index")
        return JNAL

## TYPE_PIECE

In [39]:
def TYPE_PIECE(df):
    '''Aucune valeur vide, Aucune valeur différente de "RECUP"'''
    col = "TYPE_PIECE"
    list_na = df[(df[col].isna())]["OID_ECRITURE"].tolist()
    list_not_recup=df[(df[col].notna())&(df[col]!="RECUP")]["OID_ECRITURE"].tolist()
    if len(list_na)+len(list_not_recup):

        TYPE_PIECE = pd.DataFrame.from_dict({"Contient des cellules vides":list(set(list_na)),
                                       "différent de 'RECUP'":list(set(list_not_recup))},
                                       orient="index")  
        return TYPE_PIECE

## REF PIECE

In [40]:
def contient_na(df, colonne):
    '''list les OID_ECRITURE pour lesquels la colonne spécifiée est vide'''
    col=colonne
    list_na = df[df[col].isna()]["OID_ECRITURE"].tolist()
    if len(list_na):
        dff = pd.DataFrame.from_dict({"Contient des cellules vides":list(set(list_na))})
        return dff

## DATE_PIECE

In [41]:
def DATE_PIECE(df):
    '''Aucune valeur vide, Aucune année autre que 2022'''
    list_date = []
    for date in df["DATE_PIECE"].unique():
        if str(date).split("-")[0]!="2022":
            list_date.append(df[df["DATE_PIECE"]==date]["OID_ECRITURE"].iloc[0])
    list_na = df[(df["DATE_PIECE"].isna())]["OID_ECRITURE"].tolist()
    if len(list_date)+len(list_na):
        DATE_PIECE = pd.DataFrame.from_dict({"Contient des cellules vides":list(set(list_na)),
            "dates différentes de 2022":list(set(list_date))}, orient="index")
        return DATE_PIECE

## LIB_PIECE


## CPT_GENERAL

In [42]:
def CPT_GENERAL(df):
    '''Aucune valeur vide - Aucune valeur ne commençant par "8" ou "9" - Aucune valeur "_COMPTE"'''
    col = "CPT_GENERAL"
    list_9 = []
    list_compte=[]
    list_na = df[df[col].isna()]["OID_ECRITURE"].to_list()
    for item in df[col]:
        if str(item).startswith('8') or str(item).startswith('9'):
            list_9.append(df[df[col]==item]["OID_ECRITURE"].iloc[0])
        elif item=="_COMPTE":
            list_compte.append(df[df[col]==item]["OID_ECRITURE"].iloc[0])
    if len(list_9)+len(list_compte)+len(list_na):
        CPT_GENERAL = pd.DataFrame.from_dict({"Contient des cellules vides":list(set(list_na)),
                                        "Commence par 9 ou 8":list(set(list_9)),
                                        "Contient '_COMPTE'":list(set(list_compte))}, orient="index")
        return CPT_GENERAL

## AUXILIAIRE_TIERS

In [43]:
def AUXILIAIRE_TIERS(df):
    '''Aucune valeur "999998" - Aucune valeur "_Collec" '''
    list_999998=[]
    list_collec=[]
    for item in df["AUXILIAIRE_TIERS"].unique():
        if item == "999998":
            list_999998 = df[df["AUXILIAIRE_TIERS"]=="999998"]["OID_ECRITURE"].tolist()
        elif item =="_Collec":
            list_collec = df[df["AUXILIAIRE_TIERS"]=="_Collec"]["OID_ECRITURE"].tolist()
    if len(list_999998)+len(list_collec):
        AUXILIAIRE_TIERS = pd.DataFrame.from_dict({"Contient '999998'": list(set(list_999998)),
                                             "Contient '_Collec'": list(set(list_collec))}, orient="index")
        return AUXILIAIRE_TIERS

## SENS

##  MONTANT_TENUE _CPTE 

In [44]:
def MONTANT(df, colonne):
    '''Aucune valeur vide - Aucun montant nul - Aucun montant négatif'''
    col = colonne
    dft=df.copy()
    dft["temp"]=dft[col].replace(",",".").astype(float)
    list_0 =[]
    list_neg =[]
    
    list_na = dft[dft[col].isna()]["OID_ECRITURE"].to_list()
    for item in dft["temp"]:
        if item==0:
            list_0.append(dft[dft["temp"]==item]["OID_ECRITURE"].iloc[0])
        elif item<0:
            list_neg.append(dft[dft["temp"]==item]["OID_ECRITURE"].iloc[0])
    if len(list_neg)+len(list_na)+len(list_0):
        MONTANT = pd.DataFrame.from_dict({"Contient des cellules vides":list(set(list_na)),
                                                "Est égal à zéro":list(set(list_0)),
                                                "Est négatif":list(set((list_neg)))}, orient="index")
        return MONTANT

## MONTANT_TRANSAC

## DEV

## SECTION_ana

In [45]:
def SECTION_ANA(df):
    '''Aucune valeur "_SAAX" - Aucune valeur "_METIERAX" - Aucune valeur "_MPAX"'''
    col = 'SECTION_ana'
    list_saax = []
    list_metierax = []
    list_mpax = []
    dft=df[df[col].notna()]
    for item in dft[col]:
        if item.startswith("_SAAX"):
            list_saax.append(dft[dft[col]==item]["OID_ECRITURE"].iloc[0])
        elif item.startswith("_METIERAX"):
            list_metierax.append(dft[dft[col]==item]["OID_ECRITURE"].iloc[0])
        elif item.startswith( "_MPAX"):
            list_mpax.append(dft[dft[col]==item]["OID_ECRITURE"].iloc[0])
    if len(list_saax)+len(list_metierax)+len(list_mpax):
        SECTION_ana = pd.DataFrame.from_dict({"Contient '_SAAX'": list(set(list_saax)),
                                         "Contient '_METIERAX'": list(set(list_metierax)),
                                          "Contient '_MPAX'": list(set(list_mpax))
                                         }, orient="index")
        return SECTION_ana

## DEV_TC

In [46]:
def DEV_TC(df):
    col="DEV_TC"
    list_different=df[(df[col].notna())&(df[col]!="EUR")]["OID_ECRITURE"].tolist()
    list_na = df[df[col].isna()]["OID_ECRITURE"].tolist()
    if len(list_na)+len(list_different):
        DEV_TC = pd.DataFrame.from_dict({"Contient des cellules vides": list(set(list_na)),
                                    "Différent de 'EUR'": list(set(list_different))}, orient="index")
        return DEV_TC

## PROFIL_TVA

In [47]:
def PROFIL_TVA(df):
    PROFIL_TVA = df[df["PROFIL_TVA"].notna()]["OID_ECRITURE"].tolist()
    if len(PROFIL_TVA):
        dff = pd.DataFrame.from_dict({"Contient des cellules non-vides":list(set(PROFIL_TVA))})
        return dff
        

## DETAIL_AFFAIRE

In [48]:
def DETAIL_AFFAIRE(df):
    '''Aucune valeur "_DETAFF"'''
    col = 'DETAIL_AFFAIRE'
    if len(df[df[col] == "_DETAFF"]):
        DETAIL_AFFAIRE = pd.DataFrame.from_dict({"Contient '_DETAFF'": list(
            set(df[df[col] == '_DETAFF']["OID_ECRITURE"].tolist()))},)
        return DETAIL_AFFAIRE

## CLASS_CPTE

In [49]:
def CLASS_CPTE(df):
    '''Aucune valeur vide - Doit être égal au premier caractère de la colonne "CPT_GENERAL"'''
    col = 'CLASSE_CPTE'
    list_na = df[df[col].isna()]["OID_ECRITURE"].tolist()
    list_erreur = []
    dft = df[df[col].notna()].reset_index()
    for i in range(len(dft)):
        if dft[col][i] != str(dft["CPT_GENERAL"][i])[0]:
            list_erreur.append(dft["OID_ECRITURE"][i])
    if len(list_na)+len(list_erreur):
        CLASS_COMPTE = pd.DataFrame.from_dict({"Contient des cellules vides":list(set(list_na)),
                                              "Erreur CLASS_CPTE": list(set(list_erreur))},
                                             orient="index")
        return CLASS_COMPTE    

## M_SIGNE_TC

## M_SIGNE_TR

In [50]:
def M_SIGNE(df, colonne):
    '''Aucune valeur vide - si la colonne "SENS" == "D" alors doit être > 0 - si la colonne "SENS"=="C" alors doit être <0 '''
    col = colonne
    list_na = df[df[col].isna()]["OID_ECRITURE"].tolist()
    list_erreur = []
    dft = df[df[col].notna()].reset_index()
    for i in range(len(dft)):
        if dft["SENS"][i]=="D":
            if dft[col][i]<0:
                list_erreur.append(dft["OID_ECRITURE"][i])
        elif dft["SENS"][i]=="C":
            if dft[col][i]>=0:
                list_erreur.append(dft["OID_ECRITURE"][i])
    if len(list_na)+len(list_erreur):
        M_SIGNE = pd.DataFrame.from_dict({"Contient des cellules vides":list(set(list_na)),
                                              "Erreur SIGNE": list(set(list_erreur))},
                                             orient="index")  
        return M_SIGNE

# Contrôle Quali

## Controle multi devise

In [51]:
def MULTI_DEV(df):
    '''controle qu'une clé multiple "ETAB" + "JNAL" + "REF_PIECE" + "DATE_PIECE" ne mene qu'à une seule devise '''
    dft = df[["ETAB", "JNAL", "REF_PIECE", "DATE_PIECE", "DEV"]].groupby(
        ["ETAB", "JNAL", "DATE_PIECE", "REF_PIECE"]).agg({"DEV": lambda x: x.nunique()}).reset_index()
    list_multi_dev = []
    for i in dft[dft["DEV"] > 1].index:
        list_multi_dev.append(str(dft.loc[i, "ETAB"])+"|"+dft.loc[i, "JNAL"]+"|" +
                         str(dft.loc[i, "DATE_PIECE"]).split(" ")[0]+"|"+dft.loc[i, "REF_PIECE"])
    if list_multi_dev:
        MULTI_DEV = pd.DataFrame.from_dict(
            {"Contient des devises multiple:": list_multi_dev})
        return MULTI_DEV





## Controle équilibre

In [52]:
def EQUILIBRE(df, colonne):
    '''vérifie que la somme des débit "D" de la colonne sont egaux aux crédit "C"'''
    col = colonne
    dft = df.pivot_table(index=["ETAB", "JNAL", "REF_PIECE", "DATE_PIECE"],
                         values=col, columns="SENS", aggfunc='sum').reset_index()
    dft["diff"] = round(dft['C']-dft['D'],5)
    list_eq = []
    for i in range(len(dft)):
        if dft["diff"][i] != 0:
            list_eq.append(str(dft["ETAB"][i])+"|"+str(dft["JNAL"][i])+"|"+str(
                dft["DATE_PIECE"][i]).split(" ")[0]+"|"+str(dft["REF_PIECE"][i]))
    if list_eq:
        EQUILIBRE = pd.DataFrame.from_dict({"Non équilibré": list_eq})
        return EQUILIBRE

## Controle devise ISO

In [53]:
def DEV_ISO(df):
    '''vérifie que les valeurs DEV sont bien dans la liste ISO_4217'''
    # récupère la liste des code ISO de devise depuis wikipedia
    url = "https://fr.wikipedia.org/wiki/ISO_4217"
    scrap = requests.get(url).text
    soup = BeautifulSoup(scrap, 'html.parser')
    table = soup.find("table")
    codes_iso =[]
    for row in table.find_all("tr"):
        columns = row.find_all('td')
        if columns:
            code = columns[0]
            codes_iso.append(re.sub(r"[^A-Z]","",str(code))[0:3])   
    # vérifie que les codes DEV utilisés sont bien dans la liste
    list_erreur_encodage_devise = []
    for dev in df["DEV"]:
        if dev not in codes_iso:
            list_erreur_encodage_devise.append(df.loc[df["DEV"]==dev,"OID_ECRITURE"].tolist() )
    # génére un df si il y a des erreurs
    if len(list_erreur_encodage_devise):
        DEV_ISO =pd.DataFrame.from_dict({"Devise mal encodée": list_erreur_encodage_devise}) 
        return DEV_ISO

# Génération TCD

## DELETE_SHEET

In [54]:
def delete_sheet(sheet):
    ''' efface la feuille en conservant le header'''
    while(sheet.max_row>1):
        sheet.delete_rows(2)
    return



# Generalisation

In [55]:
def controles(file, date):
    ''' effectue une série de controle sur le df et écrit un fichier excel avec un onglet par colonne 
    où se trouve une erreur et la OID de l'erreur et renvoie "KO" - Si le fichier ne contient pas d erreur renvoie "OK"'''
    
    # chargement du fichier
    #------------------------------------------------
    dtype = {'ETAB': object, 'JNAL': str, 'TYPE_PIECE': str, 'REF_PIECE': str, 'LIB_PIECE': str,
             'CPT_GENERAL': str, 'AUXILIAIRE_TIERS': object, 'LIBELLE_ECR': str, 'SENS': str,
             'MONTANT_TENUE_CPTE': object, 'MONTANT_TRANSAC': object, 'DEV': str, 'DATE_ECH': object, 'MODE_REG': object,
             'SECTION_ana': object, 'DEV_TC': object, 'CODE_LET': object, 'PROFIL_TVA': object, 'DETAIL_AFFAIRE': object,
             'TYPE_CPTE': object, 'OID_ECRITURE': object, 'CLASSE_CPTE': str}

    df = pd.read_excel(file, dtype=dtype, decimal=',')
    racine = file.split("\\")[1].split(".")[0]
   
    # controles
    #---------------------------------------------------
    etab = ETAB(df)
    jnal = JNAL(df)
    type_piece = TYPE_PIECE(df)
    ref_piece = contient_na(df, 'REF_PIECE')
    date_piece = DATE_PIECE(df)
    lib_piece = contient_na(df, 'LIB_PIECE')
    cpt_general = CPT_GENERAL(df)
    auxiliaire_tiers = AUXILIAIRE_TIERS(df)
    sens = contient_na(df, 'SENS')
    montant_tenue_cpte = MONTANT(df, 'MONTANT_TENUE_CPTE')
    montant_transac = MONTANT(df, 'MONTANT_TRANSAC')
    dev = contient_na(df, 'DEV')
    section_ana = SECTION_ANA(df)
    dev_tc = DEV_TC(df)
    profil_tva = PROFIL_TVA(df)
    detail_affaire = DETAIL_AFFAIRE(df)
    class_cpte = CLASS_CPTE(df)
    montant_signe_tc = M_SIGNE(df, "M_SIGNE_TC")
    montant_signe_tr = M_SIGNE(df, "M_SIGNE_TR")
    multi_dev = MULTI_DEV(df)
    equilibre_tenue_cpte = EQUILIBRE(df, 'MONTANT_TENUE_CPTE')
    equilibre_transac = EQUILIBRE(df, 'MONTANT_TRANSAC')
    dev_iso = DEV_ISO(df)

    all_test = [etab, jnal,type_piece,ref_piece,date_piece,lib_piece,cpt_general,auxiliaire_tiers,sens,montant_tenue_cpte,
               montant_transac, montant_signe_tc, montant_signe_tr, dev,section_ana,dev_tc,profil_tva, detail_affaire,
               class_cpte,multi_dev,equilibre_tenue_cpte,equilibre_transac,dev_iso]
    i=0
    for test in all_test:
        if test is None:
            i=i+1
    # si tous les controles sont OK on génére les tcd 
    #-------------------------------------------------------
    #if etab is None and jnal is None and type_piece is None and ref_piece is None and date_piece is None and lib_piece is None and cpt_general is None and auxiliaire_tiers is None and sens is None and montant_tenue_cpte is None and montant_transac is None and dev is None and section_ana is None and dev_tc is None and profil_tva is None and detail_affaire is None and class_cpte is None and montant_signe_tc is None and montant_signe_tr is None and multi_dev is None and equilibre_tenue_cpte is None and equilibre_transac is None and dev_iso is None:
    if len(all_test) == i:
        file_OK = "OK"
        # On charge le fichier contenant les tcd à reproduire
        wb_destination = load_workbook("fichier_destination.xlsx")
        sheet = wb_destination["Sources"]
        # On efface la feuille source
        delete_sheet(sheet)
        date_style = NamedStyle(name='date_style',number_format='dd/mm/yy')
        wb_destination.add_named_style(date_style)
        # On remplace par les données issues du df
        for row in dataframe_to_rows(df, index=False, header=False):
            sheet.append(row)
        for row in sheet.iter_rows(min_col=5, max_col=5):
            for cell in row:
                cell.style = date_style
        nom_fichier_final = "{}_TCD_{}.xlsx".format(racine, date)
        wb_destination.save("TCD\{}".format(nom_fichier_final))
    
    # si au moins une erreur a été trouvé on génére le fichier excel erreurs 
    #----------------------------------------------------------
    else:
        file_OK = "KO"
        with pd.ExcelWriter("controle\\{}_erreurs_{}.xlsx".format(racine, date)) as writer:
            if etab is not None:
                etab.T.to_excel(writer, sheet_name="ETAB", index=False)
            if jnal is not None:
                jnal.T.to_excel(writer, sheet_name="JNAL", index=False)
            if type_piece is not None:
                type_piece.T.to_excel(
                    writer, sheet_name="TYPE_PIECE", index=False)
            if ref_piece is not None:
                ref_piece.to_excel(writer, sheet_name="REF_PIECE", index=False)
            if date_piece is not None:
                date_piece.T.to_excel(
                    writer, sheet_name="DATE_PIECE", index=False)
            if lib_piece is not None:
                lib_piece.to_excel(writer, sheet_name="LIB_PIECE", index=False)
            if cpt_general is not None:
                cpt_general.T.to_excel(
                    writer, sheet_name="CPTE_GENERAL", index=False)
            if auxiliaire_tiers is not None:
                auxiliaire_tiers.T.to_excel(
                    writer, sheet_name="AUXILIAIRE_TIERS", index=False)
            if sens is not None:
                sens.to_excel(writer, sheet_name="SENS", index=False)
            if montant_tenue_cpte is not None:
                montant_tenue_cpte.T.to_excel(
                    writer, sheet_name="MONTANT_TENUE_CPTE", index=False)
            if montant_transac is not None:
                montant_transac.T.to_excel(
                    writer, sheet_name="MONTANT_TRANSAC", index=False)
            if dev is not None:
                dev.to_excel(writer, sheet_name="DEV", index=False)
            if section_ana is not None:
                section_ana.T.to_excel(
                    writer, sheet_name="SECTION_ana", index=False)
            if dev_tc is not None:
                dev_tc.to_excel(writer, sheet_name="DEV_TC", index=False)
            if profil_tva is not None:
                profil_tva.T.to_excel(
                    writer, sheet_name="PROFIL_TVA", index=False)
            if detail_affaire is not None:
                detail_affaire.to_excel(
                    writer, sheet_name="DETAIL_AFFAIRE", index=False)
            if class_cpte is not None:
                class_cpte.T.to_excel(
                    writer, sheet_name="CLASS_CPTE", index=False)
            if montant_signe_tc is not None:
                montant_signe_tc.T.to_excel(
                    writer, sheet_name="M_SIGNE_TC", index=False)
            if montant_signe_tr is not None:
                montant_signe_tr.T.to_excel(
                    writer, sheet_name="M_SIGNE_TR", index=False)
            if multi_dev is not None:
                multi_dev.to_excel(writer, sheet_name="MULTI_DEV", index=False)
            if equilibre_tenue_cpte is not None:
                equilibre_tenue_cpte.to_excel(
                    writer, sheet_name="EQUILIBRE_TENUE_CPTE", index=False)
            if equilibre_transac is not None:
                equilibre_transac.to_excel(
                    writer, sheet_name="EQUILIBRE_TRANSAC", index=False)
            if dev_iso is not None:
                dev_iso.to_excel(
                    writer, sheet_name="DEV_ISO", index=False)

    return file_OK

In [56]:
def controle_final(filepath, date):
    dtype = {'ETAB': object, 'JNAL': str, 'TYPE_PIECE': str, 'REF_PIECE': str, 'DATE_PIECE': str, 'LIB_PIECE': str,
       'CPT_GENERAL': str, 'AUXILIAIRE_TIERS': object, 'LIBELLE_ECR': str, 'SENS': str,
       'MONTANT_TENUE_CPTE': object, 'MONTANT_TRANSAC': object, 'DEV': str, 'DATE_ECH': object, 'MODE_REG': object,
       'SECTION_ana': object, 'DEV_TC': object, 'CODE_LET': object, 'PROFIL_TVA': object, 'DETAIL_AFFAIRE': object,
       'TYPE_CPTE': object, 'OID_ECRITURE': object, 'CLASSE_CPTE': str}
    batch = glob.glob("{}/CONTROLE*.xlsx".format(filepath))
    synthese = []
    date_du_jour = date
    for file in batch:
        synthese.append([file.split("\\")[1], controles(file, date_du_jour)])
    df = pd.DataFrame(synthese, columns=['Fichier testé', 'Statut'])
    with pd.ExcelWriter("synthese-RDD-{}.xlsx".format(date_du_jour)) as writer2:
        df.to_excel(writer2, sheet_name="liste", index=False)
        df.groupby(by="Statut").count().reset_index().sort_values("Statut").to_excel(writer2, sheet_name="resume", index=False)
    wb=load_workbook("synthese-RDD-{}.xlsx".format(date_du_jour))
    ws=wb["resume"]
    values = Reference(ws,min_col=2,max_col=2,min_row=1, max_row=3)  
    cats = Reference(ws,min_col=1, max_col=1, min_row=2, max_row=3)
    graph = BarChart()
    graph.type = 'bar'
    graph.add_data(values, titles_from_data=True)
    graph.set_categories(cats)
    graph.title = "Nombre de fichier par statut"
    ws.add_chart(graph, "D2")
    wb.save("synthese-RDD-{}.xlsx".format(date_du_jour))

# Fonction

In [57]:
controle_final("Travail commun AO - Fichiers pour Sage", "25-10-18-002")

C:\Users\mario\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\mario\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\mario\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
